In [ ]:
import pandas as pd
import altair as alt
import os
from dotenv import load_dotenv

load_dotenv()

csv_path = "/Users/jawaugh/labs/data_examples/multiTimeline.csv"
chart_description = "A line-chart where the line is green and the points are Orange, and there is a title that is in Roboto font (red)"

In [ ]:
import requests

API_KEY = os.environ.get("OPENAI_API_KEY")

def get_gpt3_response(prompt, stop=None):
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {"prompt": prompt, "max_tokens": 500, "temperature": 0, "model": "text-davinci-002"}
    if stop:
        data["stop"] = stop
    response = requests.post("https://api.openai.com/v1/completions", headers=headers, json=data)
    return response.json()

def get_summary_of_frame(df):
    summary_df = pd.DataFrame(df.dtypes,columns=['dtype'])
    summary_df.columns.name = 'Variable'
    summary_df['Missing values'] = df.isna().sum()
    summary_df['Uniques'] = df.nunique()
    summary_df['First value'] = df.loc[0]
    summary_df['Min value'] = df.min()
    summary_df['Max value'] = df.max()
    return summary_df

def get_prompt_for_chart(chart_description, data_summary):
    prompt_for_chart = f"""
A python function for generating a { chart_description } chart with altair.

# Explore the data
{data_summary}

CODE:
```
import altair as alt

def get_chart(df):
"""
    return prompt_for_chart

def get_chart_code(prompt, stop="```"):
    response = get_gpt3_response(prompt, stop)
    return "def get_chart(df):\n" + response["choices"][0]["text"]



In [ ]:
df = pd.read_csv(csv_path)
summary = get_summary_of_frame(df).to_string()
prompt = get_prompt_for_chart(chart_description, summary)

In [ ]:
result = get_chart_code(prompt)

In [ ]:
exec(result)
chart = get_chart(df)

In [ ]:
chart.interactive()

In [ ]:
print(result)